# Исследование надежности заемщиков

## Оглавление:
* [Изучение общей информации о данных](#и_о)
* [Предобработка данных](#п_д)
    *  [Предобработка по столбцам](#п_п)
    *  [Удаление дубликатов](#у_д)
    *  [Замена типов данных](#з_т)
    *  [Категоризация данных](#к_д)
* [Исследование данных](#и_д)
    * [Есть ли зависимость между количеством детей и возвратом кредита в срок?](#к_де)
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#с_п)
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#у_до)
    * [Как разные цели кредита влияют на его возврат в срок?](#ц_к)
* [Общий вывод ](#о_в)

## Изучение общей информации о данных <a class="anchor" id="и_о"></a>

Подключим нужные библиотеки и выведем общую информацию о данных.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Выводы**

Почти в каждом столбце сразу видны недочёты, на которые необходимо обратить внимание и исправить. Рассмотрим каждый столбец:
* `children` - минимальное значение `-1`, максимальное значение `20`, что похоже на ошибку. Нужно посмотреть на распределение данных;
* `days_employed` - максимальное значение `401755`, что составляет `1100` ktn, что очевидно является ошибкой. Так же наблюдается большое количество отрицательных значений. Есть пропуски;
* `dob_year` - минильный возраст равен `0`, что так же является ошибкой;
* `education` - нужно проверить данный столбец на правильность написания категорий;
* `education_id` - в этом столбце с данными всё в порядке. Нужно проверить соответствие `id` столбцу `education`;
* `family_status` - нужно проверить данные;
* `family_status_id` - Так же нужно проверить соответствие `id` столбцу `family_status`;
* `gender` - проверить, сколько гендеров в данном столбце;
* `income_type` - нужно проверить данный столбец на правильность написания категорий;
* `debt` - наш целевой признак, следует проверить, чтобы он принимал значения только `0` и `1`.
* `total_income` - со значениями всё в порядке, но есть пропуски. Так же для анализа удобно выделить несколько категорий по доходу;
* `purpose` - видимо в этом столбце много уникальных значений, поэтому стоит выделить основные категории.

Проведём предобработку по каждому столбцу, потом удалим дубликаты, затем скорректируем типы данных и уточним категории в `income_type` и `purpose`.

Приступим к предобработке данных.

## Предобработка данных <a class="anchor" id="п_д"></a>

### Предобработка по столбцам <a class="anchor" id="п_п"></a>

Начнём со столбца `children`.

In [5]:
data.children.value_counts()

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

Предположим, что `-1` - это ошибочное написание `1`, а `20` - это `2` и произведём соответствующие замены. 

In [6]:
data.children = data.children.apply(lambda x: 2 if x == 20 else (1 if x == -1 else x))
data.children.value_counts()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: count, dtype: int64

Теперь приступим к `days_employed` и `total_income`.

In [7]:
data.groupby('income_type')[['days_employed','total_income']].agg(['min', 'max', 'mean', 'count'])

days_employed                                       \
                           min            max           mean  count   
income_type                                                           
безработный      337524.466835  395302.838654  366413.652744      2   
в декрете         -3296.759962   -3296.759962   -3296.759962      1   
госслужащий      -15193.032201     -39.954170   -3399.896902   1312   
компаньон        -17615.563266     -30.195337   -2111.524398   4577   
пенсионер        328728.720605  401755.400475  365003.491245   3443   
предприниматель    -520.848083    -520.848083    -520.848083      1   
сотрудник        -18388.949901     -24.141633   -2326.499216  10014   
студент            -578.751554    -578.751554    -578.751554      1   

                  total_income                                      
                           min           max           mean  count  
income_type                                                         
безработный       59956.991984  2.027225e+05  131339.751676      2  
в декрете         53829.130729  5.382913e+04   53829.130729      1  
госслужащий       29200.077193  9.104515e+05  170898.309923   1312  
компаньон         28702.812889  2.265604e+06  202417.461462   4577  
пенсионер         20667.263793  7.351033e+05  137127.465690   3443  
предприниматель  499163.144947  4.991631e+05  499163.144947      1  
сотрудник         21367.648356  1.726276e+06  161380.260488  10014  
студент           98201.625314  9.820163e+04   98201.625314      1

Все большие значения `days_employed` относятся к категориям `безработный` и `пенсионер`, причём среднее значение близко к `1000` годам. При записи клиентов без работы над данный момент программа добавила три порядка? Необходимо сообщить разработчикам базы данных. Однако в рамках данного проекта не проверяется связь `days_employed` и благонадёжности клиентов, поэтому просто уберём знак `-` у значений в этом столбце и заполним пропуски медианой. Значения в `total_income` выглядят логично, так же заполним пропуски медианой. Стоит отдельно выделить категории `безработный`, `в декрете`, `предприниматель` и `студент` из-за малого количества клиентов, представленных в этих категориях. Если бы нужно было проверить гипотезу о зависимости благонадёжности клиентов от `total_income`, то потребовалось больше данных об этих категориях. Отметим так же, что записи в столбце `income_type` корректны.

In [8]:
data.days_employed = data.days_employed.abs()

In [9]:
data.total_income = data.total_income.fillna(data.groupby('income_type')['total_income'].transform('median'))
data.days_employed = data.days_employed.fillna(data.groupby('income_type')['days_employed'].transform('median'))

Посмотрим на нулевые значения в столбце `dob_years`.

In [10]:
data[data.dob_years == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1574.202821,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


101 значение, возможно пропуски при заполнении данных. Заполним средним по `income_type`.

In [11]:
mean_by_income_type = data.groupby("income_type")["dob_years"].mean().round(0)
data.loc[data["dob_years"] == 0, "dob_years"] = data[data["dob_years"] == 0]["income_type"].map(mean_by_income_type)

In [12]:
data.loc[[99, 149, 270, 578], :]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,59,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,40,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,40,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,59,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости


Нули заполнены корректно. Приступим к столбцам `education` и `education_id`.

In [13]:
data.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведём всё к нижнему регистру и проверим соответствие категорий.

In [14]:
data.education = data.education.apply(lambda x: x.lower())
data.education.unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [15]:
data.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  5260
1             среднее                15233
2             неоконченное высшее      744
3             начальное                282
4             ученая степень             6
Name: count, dtype: int64

У категорий немного нарушен порядок нумерации, но условные обозначения соответствуют. Проведём такой же анализ и для столбцов `family_status` и `family_status_id`.

In [16]:
data.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12380
1                 гражданский брак          4177
2                 вдовец / вдова             960
3                 в разводе                 1195
4                 Не женат / не замужем     2813
Name: count, dtype: int64

Всё в порядке. Посмотрим на `gender`.

In [17]:
data.gender.value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

`XNA` похоже на пропуск, заполним самым популярным значением - `F`.

In [18]:
data.loc[data.gender == 'XNA', 'gender'] = 'F'

In [19]:
data.gender.value_counts()

gender
F    14237
M     7288
Name: count, dtype: int64

`gender` исправили, теперь `debt`.

In [20]:
data.debt.value_counts()

debt
0    19784
1     1741
Name: count, dtype: int64

Всё в порядке, остался столбец `purpose`.

In [21]:
data.purpose.value_counts()

purpose
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образовани

Действительно много похожих категорий, в дальнейшем объединим их в несколько больших. Осталось проверить данные на последний существенный деффект: дубликаты.

### Удаление дубликатов <a class="anchor" id="у_д"></a>

In [22]:
data.duplicated().sum()

71

In [23]:
data.drop_duplicates(inplace=True)

С дубликатами разобрались. 

### Замена типов данных <a class="anchor" id="з_т"></a>

In [24]:
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


Для столбцов `children`, `dob_years`, `education_id`, `family_status_id`, `debt` подойдёт тип данных `int16`. Для `days_employed`, `total_income` лучше использовать `int64`.

In [25]:
data[['children', 'dob_years', 'education_id', 'family_status_id', 'debt']] = data[['children', 'dob_years', 'education_id', 'family_status_id', 'debt']].astype('int16')
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int16 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int16 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int16 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int16 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int16 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int16(5), int64(2), object(5)
memory usage: 1.5+ MB


Категоризируем данные в столбце `total_income` и в столбце `purpose_category` сгруппируем категории.

### Категоризация данных <a class="anchor" id="к_д"></a>

Выделем 5 категорий по перцентилям с равным шагом.

In [26]:
data['total_income'].describe(percentiles=[0.2,0.4,0.6,0.8,])

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
20%      9.853760e+04
40%      1.321344e+05
50%      1.425940e+05
60%      1.613350e+05
80%      2.146182e+05
max      2.265604e+06
Name: total_income, dtype: float64

Напишем функции для выполнения задач, добавив численные значения к категории по доходам для удобной сортировки.

In [27]:
def categorize_income(income):
    try:
        if 0 <= income <= 90000:
            return '1 очень низкий доход'
        elif 90001 <= income <= 130000:
            return '2 низкий доход'
        elif 130001 <= income <= 160000:
            return '3 средний доход'
        elif 160001 <= income <= 214000:
            return '4 высокий доход'
        elif income >= 214001:
            return '5 очень высокий доход'
    except:
        pass

In [28]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [29]:
data.total_income_category. = data.total_income.apply(categorize_income)
data.purpose_category. = data.purpose.apply(categorize_purpose)

In [30]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5 очень высокий доход,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2 низкий доход,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,3 средний доход,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5 очень высокий доход,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,3 средний доход,проведение свадьбы


Данные готовы для исследования.

## Исследование данных <a class="anchor" id="и_д"></a>

Так как вопросы похожи по структуре, то базовые ответы на них можно получить одной функцией. Напишем её.

In [31]:
def to_percent(x):
    return x.map('{:.2%}'.format)

def answer(df, col1):
    pivot = df.pivot_table(index=col1, values='debt', aggfunc=('count', 'mean'))
    pivot['percent_from_all'] = pivot['count'] / pivot['count'].sum()
    pivot[['mean', 'percent_from_all']] = pivot[['mean', 'percent_from_all']].apply(to_percent)
    display(pivot)

Посчитаем среднее значение просрочек по кредитам, чтобы использовать его в качестве референса для целевых групп.

In [32]:
data.debt.value_counts(normalize=True)

debt
0    0.91885
1    0.08115
Name: proportion, dtype: float64

**8.12%** просрочек - при оценке будем отталкиваться от этого значения. Дальнейший анализ будет делать упор на характеристику *"процент просрочек"*.

### Есть ли зависимость между количеством детей и возвратом кредита в срок? <a class="anchor" id="к_де"></a>

In [33]:
answer(data, 'children')

,count,mean,percent_from_all
children,,,
0,14091,7.54%,65.68%
1,4855,9.17%,22.63%
2,2128,9.49%,9.92%
3,330,8.18%,1.54%
4,41,9.76%,0.19%
5,9,0.00%,0.04%


**Вывод**

* Клиенты даже с одним ребёнком более чем на 1.5% чаще допускают просрочки по кредитам, чем клиенты без детей: **9.17% против 7.54%**.
* При этом стоит отметить, что при увеличении количества детей вероятность просрочки тоже возрастает вплоть до **9.76%** для клиентов с 4 детьми, однако данных о таких клиентах недостаточно для точной оценки.

Можно предположить, что сказывается доплнительная финансовая нагрузка в виде трат на содержание ребёнка. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок? <a class="anchor" id="с_п"></a>

In [34]:
answer(data, 'family_status')

,count,mean,percent_from_all
family_status,,,
Не женат / не замужем,2810,9.75%,13.10%
в разводе,1195,7.11%,5.57%
вдовец / вдова,959,6.57%,4.47%
гражданский брак,4151,9.35%,19.35%
женат / замужем,12339,7.55%,57.51%


Так же посмотрим на распределение в группах по полам.

In [35]:
data.groupby(['family_status','gender'])['debt'].agg(['mean','count'])

mean  count
family_status         gender                 
Не женат / не замужем F       0.068248   1729
                      M       0.144311   1081
в разводе             F       0.065171    936
                      M       0.092664    259
вдовец / вдова        F       0.057522    904
                      M       0.200000     55
гражданский брак      F       0.081869   2846
                      M       0.118774   1305
женат / замужем       F       0.068299   7760
                      M       0.087574   4579

Интересная закономерность: женщины чаще возвращают кредит в срок, чем мужчины. Посмотрим по всем данным.

In [38]:
data.groupby('gender')['debt'].value_counts(normalize=True)

gender  debt
F       0       0.929877
        1       0.070123
M       0       0.897376
        1       0.102624
Name: proportion, dtype: float64

**Вывод**

Видно, что:
* больше всего просрочек по кредитам имеют две категории: "гражданский брак" и "Не женат / не замужем" (**9.35% и 9.75%**) или другими словами, клиенты не бывавшие в браке.
* клиенты, чей брак по тем или иным причинам окончен, "вдовцы / вдовы" и "в разводе" отдают кредиты в срок значительно чаще: **6.57% и 7.11%**;
* клиенты в браке находятся "по середние", но всё же лучше среднего по всей выборке: **7.55% против 8.12%**.

Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке. Согласно данным такие клиенты менее ответственные. Получается, что узаконенные отношения ведут к более ответственному отношению к своим обязательствам - семейные люди более скрупулёзно берегут свой семейное спокойствие и несут за него ответственность. Однако самыми благонадёжными клиентами являются клиенты-одиночки. 

При группировки данных по `gender` наблюдается, что:
* во всех категориях столбца `family_status` женщины чаще мужчин возвращают кредит в срок, при сравнении по всем данным на **3%** чаще;
* самая большая разница в категории "вдовец / вдова": **5.75%** у женщин против **20%** у мужчин.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a class="anchor" id="у_до"></a>

In [36]:
answer(data, 'total_income_category')

,count,mean,percent_from_all
total_income_category,,,
1 очень низкий доход,3348,7.83%,15.61%
2 низкий доход,4981,8.43%,23.22%
3 средний доход,4405,8.88%,20.53%
4 высокий доход,4405,8.26%,20.53%
5 очень высокий доход,4315,7.05%,20.11%


**Вывод**

Получен "колокол": клиенты с доходом от 130 001 до 160 000 чаще других допускают просрочки по кредиту: **8.88%**, а самые благонадеёжные клиенты с доходом свыше 214001: **7.05%**. Интересно, что клиенты с очень низким и низким доходом чаще клиентов со средним доходом возвращают кредиты в срок. Возможно это обусловлено более серъёзным подходом к тратам. При этом разброс от среднего значения **8.12%** меньше, чем в других группах, то есть можно сказать, что доход меньше других характеристик влияет на возврат кредитов в срок.

###  Как разные цели кредита влияют на его возврат в срок? <a class="anchor" id="ц_к"></a>

In [37]:
answer(data, 'purpose_category')

,count,mean,percent_from_all
purpose_category,,,
операции с автомобилем,4306,9.36%,20.07%
операции с недвижимостью,10811,7.23%,50.39%
получение образования,4013,9.22%,18.71%
проведение свадьбы,2324,8.00%,10.83%


**Вывод**

* Лучше всех возвращают кредиты те заёмщики, которые взяли их для операций с недвижимостью: **7.23% просрчоек**
* Хуже всех в этой категории выступают заёмщики, берущие кредиты на операции с автомобилем: **9.36% просрочек**.

## Общий вывод <a class="anchor" id="о_в"></a>

**Зависимость между количеством детей и возвратом кредита в срок**

*Краткий вывод: с ростом количества детей, возврастает вероятность просрочки.*

Клиенты без детей имеют задолженности в **7.54%** случаев, наличие даже одного ребёнка резко повышает шансы до **9.17%**. С увеличением количества детей вероятность просрочки немного возрастает.

**Зависимость между семейным положением и возвратом кредита в срок**

*Краткий вывод: те, кто никогда не были в браке хуже отдают кредиты*

Распределение по категориям:
* хуже отдают кредиты клиенты со статусом "гражданский брак" и "Не женат / не замужем": **9.35%** и **9.75%**;
* лучше "вдовцы / вдовы" и "в разводе": **6.57%** и **7.11%**;
* "в браке" "по середние": **7.55%**.

При анализе по полу получили:
* во всех категориях столбца `family_status` женщины чаще мужчин возвращают кредит в срок, при сравнении по всем данным на **3%** чаще;
* самая большая разница в категории "вдовец / вдова": **5.75%** против **20%**.

**Зависимость между уровнем дохода и возвратом кредита в срок**

*Краткий вывод: клиенты со средним доходом хуже возвращают кредиты. Доход меньше других характеристик влияет на возврат кредитов в срок*

* клиенты с доходом от 130 001 до 160 000 чаще других допускают просрочки по кредиту: **8.88%**;
* самые благонадеёжные клиенты с доходом свыше 214001: **7.05%** просрочек.
* разброс от среднего значения **8.12%** в этой категории меньше, чем в других группах.

**Влияение разных целей кредита на его возврат в срок**

* лучше возвращают кредиты те заёмщики, которые взяли их для операций с недвижимостью: **7.23%** просрчоек;
* хуже в этой категории выступают заёмщики, берущие кредиты на операции с автомобилем: **9.36%** просрочек.
